In [ ]:
import sys, importlib as impL
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/vae_torch')
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery')
import vae_torch_model as vtm
import vae_torch as vt
#from data_classes import khs_dataset_v2
import vae_scripts as vs
import sae_torch as st
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import vae_utils as vu
import pandas as pd
import helperFuncs as funcH

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option("display.precision", 3)
pd.set_option('display.max_rows', None)

In [ ]:
def get_acc_err(experiments_folder, exp_base_name,cf_int,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=1):
    ae_fold_name = os.path.join(experiments_folder, exp_base_name + str(cf_int).zfill(z_fill_int))
    ae_f_name = os.path.join(ae_fold_name, ae_f_name_base)
    vfz = np.load(ae_f_name, allow_pickle=True)
    loss_log_dict = {}
    n = 0
    loss_log_dict[data_log_key] = vfz.item().get(data_log_key)
    if loss_log_dict[data_log_key] is None:
        print("cf("+str(cf_int)+") --> loss_log_dict"+str(data_log_key)+"] is none")
        return None, None, None
    n = len(loss_log_dict[data_log_key])
    if verbose>0:
        print(str(cf_int), ', ', data_log_key, " - log is loaded with len: ", n)
    if loss_key not in loss_log_dict[data_log_key][0].keys():
        return n, None, None

    if loss_key in loss_log_dict[data_log_key][0]:
        vec_len = len(loss_log_dict[data_log_key])
        los_vec_cur = [loss_log_dict[data_log_key][l][loss_key] for l in range(0, vec_len)]
        label_str = str(cf_int) + '_' + data_log_key + '_' + loss_key
        if verbose>0:
            print(label_str, los_vec_cur[-3:], "\nmax({:4.2f}),min({:4.2f})".format(np.max(los_vec_cur), np.min(los_vec_cur)))
        return n, np.min(los_vec_cur), np.max(los_vec_cur)
    return None, None, None

In [ ]:
impL.reload(vu)
experiments_folder='/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/experiments/FM/'
config_folder = '/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/configs/'
z_fill_int=2
loss_key='reconstruction' # sparsity, bottleneck_kmeans, bottleneck_act
data_log_keys=['tr_te']  # ['tr_va', 'va', 'te']
exp_base_name='exp_conv_ae_simple_is28_cf'
ae_f_name_base='ae_ft_conv_ae_simple_is28.npy'
cf_int_arr=[201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218]
#cf_int_arr=[201,204,205,207,213,214]
#cf_int_arr=[201,204,205,216,217,218]
df1 = pd.DataFrame(index=cf_int_arr, columns=['spMethod','spW','spR','kl_div','bact','lact','rcErr','lr','rcRed'])
df2 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
df3 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
for i in range(0,len(cf_int_arr)):
    cf = cf_int_arr[i]
    #load model yaml as config
    config_file = os.path.join(config_folder, 'conf_autoencoder_'+str(cf)+'.yaml')
    CONF_PARAMS_ = funcH.CustomConfigParser(config_file=config_file)
    df1.iloc[i]['spW'] = float(CONF_PARAMS_.MODEL.SPARSITY_WEIGHT)
    if df1.iloc[i]['spW'] > 0:
        df1.iloc[i]['spMethod'] = CONF_PARAMS_.MODEL.SPARSITY_ERROR
        df1.iloc[i]['spR'] = CONF_PARAMS_.MODEL.SPARSITY_REDUCTION
    else:
        df1.iloc[i]['spW'] = None
    df1.iloc[i]['kl_div'] = CONF_PARAMS_.MODEL.KL_DIV_RHO if 'kl_' in str(CONF_PARAMS_.MODEL.SPARSITY_ERROR) else None
    try:
        df1.iloc[i]['bact'] = (CONF_PARAMS_.MODEL.LAYERS.encoder.l04_act).replace('type: ','').replace(',dim:','')
    except:
        pass
    try:
        df1.iloc[i]['lact'] = CONF_PARAMS_.MODEL.LAYERS.decoder.l01_act.replace('type: ','')
    except:
        pass
    df1.iloc[i]['rcErr'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_FUNCTION
    df1.iloc[i]['lr'] = CONF_PARAMS_.MODEL.LEARNING_RATE
    df1.iloc[i]['rcRed'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_REDUCTION
    
    
    data_log_key = 'tr_te'
    loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
    n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['n'] = n

    df2.iloc[i]['rErrMin'] = mn
    df2.iloc[i]['rErrMax'] = mx
    
    loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['kmMin'] = mn
    df2.iloc[i]['kmMax'] = mx
    
    loss_key = 'bottleneck_act' #  sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['bActMin'] = mn
    df2.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'

    loss_key = 'sparsity' #  
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['bErrMin'] = mn
    df2.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'

    
    data_log_key = 'te'
    loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
    n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['n'] = n

    df3.iloc[i]['rErrMin'] = mn
    df3.iloc[i]['rErrMax'] = mx
    
    loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['kmMin'] = mn
    df3.iloc[i]['kmMax'] = mx
    
    loss_key = 'bottleneck_act' #  sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['bActMin'] = mn
    df3.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'

    loss_key = 'sparsity' #  
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['bErrMin'] = mn
    df3.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'

In [ ]:
pd.set_option("display.precision", 2)
print('experiments explanations')
print(df1)

In [ ]:
print('tr_te dataset results')
pd.set_option("display.precision", 5)
print(df2)

In [ ]:
print('te dataset results')
print(df3)